# Feature Selection using BPSO for SDP

installations

In [1]:
 pip install pyswarms

  Stored in directory: C:\Users\afnan\AppData\Local\pip\Cache\wheels\8b\99\a0\81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


import libraries 

In [11]:
import numpy as np
import seaborn as sns
import pandas as pd
import os
import pyswarms as ps
from scipy.io import arff

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### The methods 

Dataset prepration

In [13]:
def data_prepration(file):
    data = arff.loadarff(file)
    X=[]
    y=[]
    for i in data[0]:

        X.append(list(i)[0:21])
        if(i[21] == b'false'):
            y.append(0)
        else:
            y.append(1)
        
    X=np.array(X)
    y=np.array(y)
    return X,y

plot data per feature (each class with different color)

In [3]:
def plot_features(features,labels):
    df = pd.DataFrame(features)
    df['labels'] = pd.Series(labels)

    sns.pairplot(df, hue='labels' , diag_kind='hist')

fitness function for each particle

In [5]:
def f_per_particle(m):
    classifier = KNeighborsClassifier(n_neighbors=5)
    """Computes for the fitness function per particle

    Inputs
    ------
    m : numpy.ndarray
        Binary mask that can be obtained from BinaryPSO, will
        be used to mask features.

    Returns
    -------
    numpy.ndarray
        Computed finess function
    """
    
    # Get the subset of the features from the binary mask
    if np.count_nonzero(m) == 0:
        X_subset = X_train
        X_subset_test= X_test
    else:
        X_subset = X_train[:,m==1]
        X_subset_test= X_test[:,m==1]
        
    
    # Perform classification and compute the fitness function
    classifier.fit(X_subset, y_train)
    j = (classifier.predict(X_subset_test) != y_test).mean() + np.count_nonzero(m)
#     j=np.count_nonzero(m)

    return j

Define fitness function for whole swarm

In [6]:
def f(x):
    """Higher-level method to do classification in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [f_per_particle(x[i]) for i in range(n_particles)]
    
    return np.array(j)

select the features from the set of features X using BPSO on the specfied parameters

In [7]:
def select_features(nparticles):
  
    total_features = X.shape[1]
    
    dimensions = total_features
    
    optimizer = ps.discrete.BinaryPSO(n_particles=nparticles, dimensions=dimensions, options=options)
    
    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=iters)
    
    
    print("cost: "+str(cost))
    fr.write(clas+"-"+ds+" dataset:\ncost= "+str(cost)+"\nNumber of selected features:"+str(len([i for i in pos if i==1]))+"\nSelected features:"+str(pos))
    return cost, pos

Train clssifier on the selected features (corrposining to pos which =1)

In [8]:
def train_and_test(clas,X_train,X_test,y_train,y_test,pos, fr):   
    
    if(clas=='NB'):
        classifier = GaussianNB()
    elif(clas=='SVM'):
        classifier = svm.SVC(kernel='linear')
    elif(clas=='KNN'):
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif(clas=='Linear Regression'):
        classfier = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
        
    # Get the selected features from the final positions
    X_selected_features = X_train[:,pos==1]  # subset
   
    # Perform classification and store performance in P
    classifier.fit(X_selected_features,y_train)
    selected_test= X_test[:,pos==1]
    
    y_pred= classifier.predict(selected_test)    
    report=classification_report(y_test,y_pred)
    subset_performance = (y_pred== y_test).mean()

    # Compute performance
#     subset_performance = (classifier.predict(selected_test) == y_test).mean()
    fr.write("\nsubset_performance:"+str(subset_performance)+"\n")
    fr.write("\nclassification_report:"+str(report)+"\n")
    
    print('Subset performance: %.3f' % (subset_performance))

## The main cell 

here the classifier is created, options, number of iterations are set, and other methods are called

each results for each dataset are stored in results folder with file of the classifier name

In [19]:
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# clas='NB'

# from sklearn import svm
# classifier = svm.SVC(kernel='linear')
# clas='SVM linear'

# from sklearn import svm
# classifier = svm.SVC(kernel='rbf')
# clas='SVM RBF'

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
clas='KNN'

# from sklearn import linear_model
# classifier = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
# clas='Linear Regression'

options = {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k':20, 'p':2}
# k : number of neighbors to be considered. Must be a positive integer less than n_partcles 
# p: the Minkowski p-norm to use. 1 is the sum-of-absolute values (or L1 distance) while 2 is the Euclidean (or L2) distance
iters= 100


root_folder="D:\\SDP\\Afnan"
files = [os.path.join(root_folder, x) for x in os.listdir(root_folder)]
for file in files :
    start= time.time()
    
    ds=file.split("\\")[3][0:-5]
    fr = open("Test/BPSO/"+clas+"/{}.txt".format(ds), "a")
    fr.write('parameters: '+str(options)+"\n")
    
    X,y = data_prepration(file)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    cost, pos = select_features(20)
    train_and_test(clas,X_train,X_test,y_train,y_test,pos,fr)
    period= time.time() - start 
    fr.write('number of particle {}:\n'.format(20))
    fr.write("best pos cost= "+str(cost)+"\nNumber of selected features:"+str(len([i for i in pos if i==1]))+"\nSelected features:"+str(pos))
    
    fr.write('\nFS, training, and testing time= '+str(period)+" seconds\n")
    fr.close()

2021-12-01 20:11:56,793 - pyswarms.discrete.binary - INFO - Optimize for 100 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k': 20, 'p': 2}

pyswarms.discrete.binary:   0%|                                                                                  |0/100
pyswarms.discrete.binary:   0%|                                                                     |0/100, best_cost=7
pyswarms.discrete.binary:   0%|                                                                     |0/100, best_cost=7
pyswarms.discrete.binary:   2%|█▍                                                                   |2/100, best_cost=7
pyswarms.discrete.binary:   2%|█▍                                                                   |2/100, best_cost=7
pyswarms.discrete.binary:   2%|█▍                                                                   |2/100, best_cost=7
pyswarms.discrete.binary:   4%|██▊                                                                  |4/100, best_cost=7
pyswarms.discrete.bin

pyswarms.discrete.binary:  42%|████████████████████████████▌                                       |42/100, best_cost=6
pyswarms.discrete.binary:  44%|█████████████████████████████▉                                      |44/100, best_cost=6
pyswarms.discrete.binary:  44%|█████████████████████████████▉                                      |44/100, best_cost=6
pyswarms.discrete.binary:  44%|█████████████████████████████▉                                      |44/100, best_cost=6
pyswarms.discrete.binary:  46%|███████████████████████████████▎                                    |46/100, best_cost=6
pyswarms.discrete.binary:  46%|███████████████████████████████▎                                    |46/100, best_cost=6
pyswarms.discrete.binary:  46%|███████████████████████████████▎                                    |46/100, best_cost=6
pyswarms.discrete.binary:  48%|████████████████████████████████▋                                   |48/100, best_cost=6
pyswarms.discrete.binary:  48%|█████████

pyswarms.discrete.binary:  86%|██████████████████████████████████████████████████████████▍         |86/100, best_cost=5
pyswarms.discrete.binary:  88%|███████████████████████████████████████████████████████████▊        |88/100, best_cost=5
pyswarms.discrete.binary:  88%|███████████████████████████████████████████████████████████▊        |88/100, best_cost=5
pyswarms.discrete.binary:  88%|███████████████████████████████████████████████████████████▊        |88/100, best_cost=5
pyswarms.discrete.binary:  90%|█████████████████████████████████████████████████████████████▏      |90/100, best_cost=5
pyswarms.discrete.binary:  90%|█████████████████████████████████████████████████████████████▏      |90/100, best_cost=5
pyswarms.discrete.binary:  90%|█████████████████████████████████████████████████████████████▏      |90/100, best_cost=5
pyswarms.discrete.binary:  92%|██████████████████████████████████████████████████████████████▌     |92/100, best_cost=5
pyswarms.discrete.binary:  92%|█████████

cost: 5.0
Subset performance: 1.000



pyswarms.discrete.binary:   1%|▋                                                                    |1/100, best_cost=6
pyswarms.discrete.binary:   2%|█▍                                                                   |2/100, best_cost=6
pyswarms.discrete.binary:   2%|█▍                                                                   |2/100, best_cost=6
pyswarms.discrete.binary:   3%|██                                                                   |3/100, best_cost=6
pyswarms.discrete.binary:   3%|██                                                                   |3/100, best_cost=6
pyswarms.discrete.binary:   4%|██▊                                                                  |4/100, best_cost=6
pyswarms.discrete.binary:   4%|██▊                                                                  |4/100, best_cost=6
pyswarms.discrete.binary:   5%|███▍                                                                 |5/100, best_cost=6
pyswarms.discrete.binary:   5%|███▍    

...


pyswarms.discrete.binary:   0%|                                                                     |0/100, best_cost=5
pyswarms.discrete.binary:   1%|▋                                                                    |1/100, best_cost=5
pyswarms.discrete.binary:   1%|▋                                                                    |1/100, best_cost=5
pyswarms.discrete.binary:   2%|█▍                                                                   |2/100, best_cost=5
pyswarms.discrete.binary:   2%|█▍                                                                   |2/100, best_cost=5
pyswarms.discrete.binary:   3%|██                                                                   |3/100, best_cost=5
pyswarms.discrete.binary:   3%|██                                                                   |3/100, best_cost=5
pyswarms.discrete.binary:   4%|██▊                                                                  |4/100, best_cost=5
pyswarms.discrete.binary:   4%|██▊     

pyswarms.discrete.binary:  33%|██████████████████████▍                                             |33/100, best_cost=4
pyswarms.discrete.binary:  34%|███████████████████████                                             |34/100, best_cost=4
pyswarms.discrete.binary:  34%|███████████████████████                                             |34/100, best_cost=4
pyswarms.discrete.binary:  35%|███████████████████████▊                                            |35/100, best_cost=4
pyswarms.discrete.binary:  35%|███████████████████████▊                                            |35/100, best_cost=4
pyswarms.discrete.binary:  36%|████████████████████████▍                                           |36/100, best_cost=4
pyswarms.discrete.binary:  36%|████████████████████████▍                                           |36/100, best_cost=4
pyswarms.discrete.binary:  37%|█████████████████████████▏                                          |37/100, best_cost=4
pyswarms.discrete.binary:  37%|█████████

pyswarms.discrete.binary:  66%|████████████████████████████████████████████▉                       |66/100, best_cost=4
pyswarms.discrete.binary:  67%|█████████████████████████████████████████████▌                      |67/100, best_cost=4
pyswarms.discrete.binary:  67%|█████████████████████████████████████████████▌                      |67/100, best_cost=4
pyswarms.discrete.binary:  68%|██████████████████████████████████████████████▏                     |68/100, best_cost=4
pyswarms.discrete.binary:  68%|██████████████████████████████████████████████▏                     |68/100, best_cost=4
pyswarms.discrete.binary:  69%|██████████████████████████████████████████████▉                     |69/100, best_cost=4
pyswarms.discrete.binary:  69%|██████████████████████████████████████████████▉                     |69/100, best_cost=4
pyswarms.discrete.binary:  70%|███████████████████████████████████████████████▌                    |70/100, best_cost=4
pyswarms.discrete.binary:  70%|█████████

pyswarms.discrete.binary:  99%|███████████████████████████████████████████████████████████████████▎|99/100, best_cost=4
pyswarms.discrete.binary: 100%|███████████████████████████████████████████████████████████████████|100/100, best_cost=4
2021-12-01 20:13:40,483 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 4.0, best pos: [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0]


cost: 4.0
Subset performance: 1.000


In [20]:
classifier.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [22]:
p=classifier.predict(X_test)

In [18]:
file.split("\\")[3][0:-5]

'PC2'

In [23]:
subset_performance = (p== y_test).mean()

In [24]:
subset_performance

1.0

In [37]:
    data = arff.loadarff(file)
    X=[]
    y=[]
    for i in data[0]:

        X.append(list(i)[0:21])
        if(i[len(i)-1] == b'Y'):
            y.append(1)
        else:
            y.append(0)
        
    X=np.array(X)
    y=np.array(y)


In [38]:
data[0]

array([( 3., 11., 2., 1.,  4., 16.,  6., 0.19,  8., 4., 0.67, 22.,  1., 0.  ,  31., 0.,  6., 1.  , 27.55, 17.19, 8.1423000e+03, 0.16,  89., 0.06, 4.52350e+02,  473.65, 0.17,  4.,  8., 18., 0.14,  38.,  51., 21., 19.,  43., 13.89,  32., b'Y'),
       ( 6., 17., 2., 0.,  3., 24.,  9., 0.16,  8., 4., 0.44, 26.,  6., 0.63,  57., 0.,  9., 1.  , 52.59, 24.94, 3.2717390e+04, 0.44, 229., 0.04, 1.81763e+03, 1311.69, 0.67,  8., 12., 19., 0.13,  97., 132., 35., 18.,  72.,  5.  ,  57., b'N'),
       ( 0., 11., 5., 3.,  6., 20.,  6., 0.19, 10., 6., 1.  , 24.,  1., 0.  ,  29., 0.,  6., 1.  , 42.16, 12.57, 6.6628600e+03, 0.18, 106., 0.08, 3.70160e+02,  530.  , 0.17,  5., 10., 20., 0.15,  48.,  58., 21., 11.,  40., 23.68,  32., b'N'),
       ...,
       ( 1., 33., 0., 4., 11., 40., 17., 0.39,  8., 1., 0.06, 43., 17., 1.  ,  40., 0., 17., 1.  , 33.51, 20.9 , 1.4637640e+04, 0.23, 149., 0.05, 8.13200e+02,  700.37, 1.  , 16., 24., 28., 0.26,  57.,  92., 15., 11.,  66., 27.27,  44., b'N'),
       (20., 33.

In [35]:
y

array([0, 1, 1, ..., 1, 0, 1])